<a href="https://colab.research.google.com/github/mohammadgh76/Federated-Learning/blob/main/Create_DataSet_For_NSLKDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flwr[simulation] torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 17.6 MB/s eta 0:00:00


In [3]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
# from torchvision.datasets import CIFAR10

import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.1.0+cu118 and Flower 1.6.0


In [5]:

import types
from numpy.lib.arraysetops import unique
import collections
from typing import List, Optional, Tuple


import numpy as np
import tensorflow as tf
# import tensorflow_federated as tff
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

np.random.seed(0)

trn = pd.read_csv("/content/kdd_train.csv")
trn.isnull().sum()
trn.nunique()
types = trn["labels"].unique()
types = types[1:]
trn["labels"].replace(to_replace=types,value="attacking",inplace = True)
c = len(trn.select_dtypes(include=["number"]).columns)

cols = ['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']

for col in trn.select_dtypes(include=["object"]):
    print(col)

from sklearn.preprocessing import LabelEncoder
cols = ['protocol_type','service','flag','labels']
for i in cols:
    en = LabelEncoder()
    trn[i] = en.fit_transform(trn[i])


corr_matrix = trn.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
trn.drop(to_drop, axis=1, inplace=True)

trn.head(10)
X_trn = trn.drop(['labels'] , axis = 1).values
Y_trn = trn['labels'].values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trn = scaler.fit_transform(X_trn)

protocol_type
service
flag
labels


125973

In [30]:
np.random.seed(0)

tst = pd.read_csv("/content/kdd_test.csv")
tst.isnull().sum()
tst.nunique()
types = tst["labels"].unique()
types = types[1:]
tst["labels"].replace(to_replace=types,value="attacking",inplace = True)
c = len(tst.select_dtypes(include=["number"]).columns)

cols = ['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
       'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate']

for col in tst.select_dtypes(include=["object"]):
    print(col)

from sklearn.preprocessing import LabelEncoder
cols = ['protocol_type','service','flag','labels']
for i in cols:
    en = LabelEncoder()
    tst[i] = en.fit_transform(tst[i])


corr_matrix = tst.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
tst.drop(to_drop, axis=1, inplace=True)

# X_trn = trn.drop(['labels'] , axis = 1).values
# Y_trn = trn['labels'].values
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# X_trn = scaler.fit_transform(X_trn)
tst.head(10)

trn = trn.drop_duplicates()
trn = trn.reset_index(drop=True)


tst = tst.drop_duplicates()
tst = tst.reset_index(drop=True)


protocol_type
service
flag
labels


In [66]:
def create_data_set(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    trainset = trn
    testset = tst

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    remainder = len(trainset) % num_clients

    # Add the remainder to the first partition
    lengths[0] += remainder

    # Manually create DataLoader instances for each client
    trainloaders = []
    valloaders = []
    for i in range(num_clients):
        start_idx = i * partition_size
        end_idx = start_idx + lengths[i]

        # Use iloc to select rows based on index range
        ds_train = trainset.iloc[start_idx:end_idx].reset_index(drop=True)
        len_val = len(ds_train) // 10
        len_train = len(ds_train) - len_val

        # Use iloc to split into train/val based on index range
        ds_train, ds_val = ds_train.iloc[:len_train].reset_index(drop=True), ds_train.iloc[len_train:].reset_index(drop=True)

        # Convert DataFrame to PyTorch tensors
        X_train, y_train = torch.tensor(ds_train.values), torch.tensor(ds_train['labels'].values)
        X_val, y_val = torch.tensor(ds_val.values), torch.tensor(ds_val['labels'].values)

        # Create PyTorch Dataset and DataLoader for each client
        train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
        val_dataset = torch.utils.data.TensorDataset(X_val, y_val)

        trainloaders.append(DataLoader(train_dataset, batch_size=320, shuffle=True))
        valloaders.append(DataLoader(val_dataset, batch_size=320))

    # Convert DataFrame to PyTorch tensors for the test set
    X_test, y_test = torch.tensor(testset.values), torch.tensor(testset['labels'].values)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    testloader = DataLoader(test_dataset, batch_size=320)

    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = create_data_set(NUM_CLIENTS)


In [73]:
print(next(iter(trainloaders[0])))


[tensor([[ 0.0000,  1.0000, 46.0000,  ...,  0.0000,  0.1400,  0.0000],
        [ 4.0000,  1.0000, 44.0000,  ...,  0.0000,  0.0500,  0.0000],
        [ 0.0000,  1.0000, 46.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  1.0000, 46.0000,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.0000,  0.0000, 14.0000,  ...,  0.5000,  0.0000,  0.0000],
        [ 0.0000,  1.0000, 46.0000,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64), tensor([0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 

In [69]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [70]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [74]:
trainloader = trainloaders[0]
valloader = valloaders[0]
net = Net().to(DEVICE)

for epoch in range(5):
    train(net, trainloader, 1)
    loss, accuracy = test(net, valloader)
    print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")

loss, accuracy = test(net, testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

RuntimeError: ignored